In [1]:
%matplotlib widget

import logging
import time

import cftime
import httpx
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
import shapely.io
import sqlmodel
from loess.loess_1d import loess_1d

from arpav_ppcv import (
    database as db,
    operations,
)
from arpav_ppcv.config import get_settings
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
    ObservationAggregationType,
    Season,
)
from arpav_ppcv.schemas.coverages import CoverageInternal

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = get_settings()
session = sqlmodel.Session(db.get_engine(settings))
http_client = httpx.Client()

In [2]:
coverage_identifier = "tas_seasonal_absolute_model_ensemble-rcp26-DJF"
point_coords = "POINT(11.5469 44.9524)"
date_range = "../.."

cov = CoverageInternal(
    configuration=db.get_coverage_configuration_by_coverage_identifier(session, coverage_identifier), 
    identifier=coverage_identifier
)

In [3]:
settings.debug = False

In [4]:
new_time_start = time.time()
coverage_series, observation_series = operations.get_coverage_time_series(
    settings=settings,
    session=session,
    http_client=httpx.AsyncClient(),
    coverage=cov,
    point_geom=shapely.io.from_wkt(point_coords),
    temporal_range=date_range,
    coverage_smoothing_strategies=[
        CoverageDataSmoothingStrategy.NO_SMOOTHING,
        CoverageDataSmoothingStrategy.MOVING_AVERAGE_11_YEARS,
        CoverageDataSmoothingStrategy.LOESS_SMOOTHING
    ],
    observation_smoothing_strategies=[
        ObservationDataSmoothingStrategy.NO_SMOOTHING,
        ObservationDataSmoothingStrategy.MOVING_AVERAGE_5_YEARS
    ],
    include_coverage_data=True,
    include_observation_data=False,
    include_coverage_uncertainty=True,
    include_coverage_related_data=True
)
new_time_end = time.time()
print(f"elapsed: {new_time_end - new_time_start}")

DEBUG:asyncio:Using selector: EpollSelector


elapsed: 0.49170613288879395


In [5]:
[(c.identifier, s.value) for c, s in coverage_series.keys()]

[('tas_seasonal_absolute_model_ec_earth_cclm4_8_17-rcp26-DJF', 'NO_SMOOTHING'),
 ('tas_seasonal_absolute_model_ec_earth_cclm4_8_17-rcp26-DJF',
  'MOVING_AVERAGE_11_YEARS'),
 ('tas_seasonal_absolute_model_ec_earth_cclm4_8_17-rcp26-DJF',
  'LOESS_SMOOTHING'),
 ('tas_seasonal_absolute_model_ensemble-rcp26-DJF', 'NO_SMOOTHING'),
 ('tas_seasonal_absolute_model_ensemble-rcp26-DJF', 'MOVING_AVERAGE_11_YEARS'),
 ('tas_seasonal_absolute_model_ensemble-rcp26-DJF', 'LOESS_SMOOTHING'),
 ('tas_seasonal_absolute_model_ec_earth_racmo22e-rcp26-DJF', 'NO_SMOOTHING'),
 ('tas_seasonal_absolute_model_ec_earth_racmo22e-rcp26-DJF',
  'MOVING_AVERAGE_11_YEARS'),
 ('tas_seasonal_absolute_model_ec_earth_racmo22e-rcp26-DJF',
  'LOESS_SMOOTHING'),
 ('tas_seasonal_absolute_model_ec_earth_rca4-rcp26-DJF', 'NO_SMOOTHING'),
 ('tas_seasonal_absolute_model_ec_earth_rca4-rcp26-DJF',
  'MOVING_AVERAGE_11_YEARS'),
 ('tas_seasonal_absolute_model_ec_earth_rca4-rcp26-DJF', 'LOESS_SMOOTHING'),
 ('tas_seasonal_absolute_model_

In [5]:
raw_data = await operations.retrieve_multiple_ncss_datasets(
    settings, 
    httpx.AsyncClient(), 
    [
        CoverageInternal(
            identifier=coverage_identifier, 
            configuration=db.get_coverage_configuration_by_coverage_identifier(
                session, coverage_identifier)
        ),
    ],
    shapely.io.from_wkt(point_coords),
    (None, None)
)
    

In [6]:
for cov, data_ in raw_data.items():
    df = operations._parse_ncss_dataset(data_, cov.configuration.netcdf_main_dataset_name, None, None, cov.identifier)

In [7]:
import pyloess

In [10]:
smoothed = pyloess.loess(df.index.year.astype("int").values, df[coverage_identifier], span=0.75, degree=2)

In [11]:
smoothed

array([[1976.        ,    3.39045092],
       [1977.        ,    3.41520367],
       [1978.        ,    3.43951448],
       [1979.        ,    3.46338917],
       [1980.        ,    3.48683499],
       [1981.        ,    3.50985909],
       [1982.        ,    3.53246727],
       [1983.        ,    3.55466547],
       [1984.        ,    3.57645893],
       [1985.        ,    3.59785423],
       [1986.        ,    3.6188608 ],
       [1987.        ,    3.63948834],
       [1988.        ,    3.65974729],
       [1989.        ,    3.67964993],
       [1990.        ,    3.69921081],
       [1991.        ,    3.71844372],
       [1992.        ,    3.73736291],
       [1993.        ,    3.75598335],
       [1994.        ,    3.77432163],
       [1995.        ,    3.7923959 ],
       [1996.        ,    3.81022421],
       [1997.        ,    3.82782181],
       [1998.        ,    3.8452036 ],
       [1999.        ,    3.86238304],
       [2000.        ,    3.87937367],
       [2001.        ,   

In [12]:
smoothed[:, 1]

array([3.39045092, 3.41520367, 3.43951448, 3.46338917, 3.48683499,
       3.50985909, 3.53246727, 3.55466547, 3.57645893, 3.59785423,
       3.6188608 , 3.63948834, 3.65974729, 3.67964993, 3.69921081,
       3.71844372, 3.73736291, 3.75598335, 3.77432163, 3.7923959 ,
       3.81022421, 3.82782181, 3.8452036 , 3.86238304, 3.87937367,
       3.89618642, 3.91283006, 3.92931283, 3.94564078, 3.96181679,
       3.97784126, 3.99370947, 4.00941002, 4.02492491, 4.04022963,
       4.05528821, 4.07005002, 4.08445509, 4.09843551, 4.11190995,
       4.12477485, 4.13690201, 4.14813488, 4.15831013, 4.16730638,
       4.17513242, 4.18196441, 4.19625107, 4.21062102, 4.22493585,
       4.23921175, 4.25354211, 4.26806956, 4.28300472, 4.29854456,
       4.31481714, 4.33193141, 4.34987622, 4.36855939, 4.38791159,
       4.40782196, 4.42815529, 4.44861772, 4.46891522, 4.48884042,
       4.50831509, 4.52728011, 4.54563333, 4.56325833, 4.58006323,
       4.59600293, 4.61099795, 4.6248397 , 4.63730183, 4.64824